In [25]:
import torch
from torch import nn
import torchvision
import torchvision.transforms as transforms
import numpy as np
from tqdm import tqdm
import os
import pickle
import statistics

In [26]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [27]:
torch.cuda.current_device()

0

# prepare for dataset

In [28]:
transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
)

trainset = torchvision.datasets.CIFAR10(root="../data/CIFAR10",
                                            train=True, download=True,
                                            transform=transform)
train_loader = torch.utils.data.DataLoader(trainset, batch_size=512,
                                               shuffle=True, num_workers=4, pin_memory=True)

Files already downloaded and verified


# model

## generator


In [29]:
class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        self.main = nn.Sequential(
            nn.ConvTranspose2d(128, 256, 4, 1, 0), # 4x4
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),

            nn.ConvTranspose2d(256, 128, 2, 2, 0), #8x8
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),

            nn.ConvTranspose2d(128, 64, 2, 2, 0), #16x16
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),

            nn.ConvTranspose2d(64, 3, 2, 2, 0), #32x32
            nn.Tanh()
        )

    def forward(self, x):
        return self.main(x)

## discriminator

In [30]:
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.LeakyReLU(0.2, inplace=True),

            nn.AvgPool2d(2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.2, inplace=True),

            nn.AvgPool2d(2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),

            nn.AvgPool2d(2),
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),

            nn.AvgPool2d(4),
            nn.Conv2d(256, 1, 1) # fcの代わり
        )

    def forward(self, x):
        return self.model(x).squeeze()

# prepare for train

In [31]:
device
model_g, model_d = Generator(), Discriminator()
model_g, model_d = nn.DataParallel(model_g), nn.DataParallel(model_d)
model_g, model_d = model_g.to(device), model_d.to(device)

params_g = torch.optim.Adam(model_g.parameters(),
            lr=0.0002, betas=(0.5, 0.999))
params_d = torch.optim.Adam(model_d.parameters(),
            lr=0.0002, betas=(0.5, 0.999))


# label
ones = torch.ones(512).to(device)
zeros = torch.zeros(512).to(device)
loss_f = nn.BCEWithLogitsLoss()
# エラー推移
result = {}
result["log_loss_g"] = []
result["log_loss_d"] = []

# train

In [32]:
# 訓練

for i in range(300):
    log_loss_g, log_loss_d = [], []
    for real_img, _ in tqdm(train_loader):
        batch_len = len(real_img)

        # Gの訓練
        # 偽画像を作成
        z = torch.randn(batch_len, 128, 1, 1).to(device)
        fake_img = model_g(z)

        # 偽画像を一時保存
        fake_img_tensor = fake_img.detach()

        # 偽画像を本物と騙せるようにロスを計算
        out = model_d(fake_img)
        loss_g = loss_f(out, ones[:batch_len])
        log_loss_g.append(loss_g.item())

        # 微分計算・重み更新
        params_d.zero_grad()
        params_g.zero_grad()
        loss_g.backward()
        params_g.step()

        # Discriminatoの訓練
        # sample_dataの実画像
        real_img = real_img.to(device)
        # 実画像を実画像と識別できるようにロスを計算
        real_out = model_d(real_img)
        loss_d_real = loss_f(real_out, ones[:batch_len])

        # 偽の画像の偽と識別できるようにロスを計算
        fake_out = model_d(fake_img_tensor)
        loss_d_fake = loss_f(fake_out, zeros[:batch_len])

        # 実画像と偽画像のロスを合計
        loss_d = loss_d_real + loss_d_fake
        log_loss_d.append(loss_d.item())

        # 微分計算・重み更新
        params_d.zero_grad()
        params_g.zero_grad()
        loss_d.backward()
        params_d.step()

    result["log_loss_g"].append(statistics.mean(log_loss_g))
    result["log_loss_d"].append(statistics.mean(log_loss_d))
    print("log_loss_g =", result["log_loss_g"][-1], ", log_loss_d =", result["log_loss_d"][-1])

    # 画像を保存
    if not os.path.exists("cifar_generated"):
        os.mkdir("cifar_generated")
    torchvision.utils.save_image(fake_img_tensor[:min(batch_len, 100)],
                            f"cifar_generated/epoch_{i:03}.png")
# ログの保存
with open("cifar_generated/logs.pkl", "wb") as fp:
    pickle.dump(result, fp)

100% 98/98 [00:07<00:00, 12.77it/s]


log_loss_g = 0.8045306649743295 , log_loss_d = 1.1647337212854503


100% 98/98 [00:06<00:00, 15.90it/s]


log_loss_g = 0.7755461779175973 , log_loss_d = 1.2587564952519474


100% 98/98 [00:06<00:00, 15.83it/s]


...

log_loss_g = 2.9845298400947025 , log_loss_d = 0.4639975420978604


100% 98/98 [00:06<00:00, 16.31it/s]


log_loss_g = 3.023403101125542 , log_loss_d = 0.33004281945450575


100% 98/98 [00:05<00:00, 16.43it/s]

log_loss_g = 3.2743921146100883 , log_loss_d = 0.12197011724418523


In [34]:
 import pandas as pd

df = pd.read_pickle("cifar_generated/logs.pkl")
df

{'log_loss_g': [0.8045306649743295,
  0.7755461779175973,
  0.7773324062629622,
  0.7894815522797254,
  0.822343492994503,
  0.84222924648499,
  0.8679401448794773,
  0.8776060148161284,
  0.9042877944148316,
  0.9403201560584866,
  0.9511809409881125,
  0.9735024613993508,
  0.9643630798982115,
  1.0324124401929426,
  1.0496293215119108,
  1.0688470589871308,
  1.0975324031041593,
  1.1147272167157154,
  1.137584031236415,
  1.1795576196544024,
  1.1916232759855232,
  1.1993953762005787,
  1.25194954568026,
  1.226149313303889,
  1.268004023907136,
  1.2972945747326832,
  1.318202291216169,
  1.3406694671329187,
  1.3401304684123214,
  1.3606973637123496,
  1.3793445259940869,
  1.3925450900379492,
  1.3922412468462575,
  1.429297294543714,
  1.4482128462012933,
  1.4608189305480646,
  1.4776795308808892,
  1.5540771536072906,
  1.5121714649151783,
  1.5416358466051063,
  1.596781697200269,
  1.5874838068777202,
  1.5951927504977401,
  1.5863909143574384,
  1.623551110223848,
  1.6425